In [ ]:
import cv2 
import copy
import math
import sys
import gc
import numpy as np
import pandas as pd

import os
from os import listdir

from matplotlib import pyplot as plt

In [ ]:
def count_files_number(x, y, z):
    path = x; file = y; file_1 = z
    c = 0
    for a in os.listdir(path + "/" + file + "/" + file_1):
        c = c + 1
    return(c)

In [ ]:
%%time
path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
number_of_files_Same = []
count_file_in_same_subfolder = []
for file in os.listdir(path):                                
    temp = []
    for file_1 in os.listdir(path + "/" + file): 
        count_file_in_each_folder = count_files_number(path, file, file_1)
        temp.append(count_file_in_each_folder)
    if temp[0]==temp[1] and temp[0]==temp[2] and temp[0]==temp[3]:
        number_of_files_Same.append(file)
        count_file_in_same_subfolder.append(temp)
count_file_in_same_subfolder = np.array(count_file_in_same_subfolder)

In [ ]:
del temp, file, file_1, count_files_number
gc.collect()

In [ ]:
csv_dataframe = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
label_dataframe = copy.deepcopy(csv_dataframe)

label_dataframe.insert(1, 'With_zero', np.ones(len(label_dataframe['BraTS21ID']), dtype=str))

count = 0
for i in label_dataframe['BraTS21ID']:
    if i>=0 and i<10:                
        i = '0000' + str(i)
    elif i>=10 and i<100:            
        i = '000' + str(i)
    elif i>=100 and i<1000:          
        i = '00' + str(i)
    elif i>=1000 and i<10000:        
        i = '0' + str(i)
    label_dataframe['With_zero'][count] = i
    count = count + 1

In [ ]:
del csv_dataframe, count, label_dataframe['BraTS21ID']
gc.collect()

In [ ]:
csv_dataframe = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
list_label = []
for i in range(len(label_dataframe['With_zero'])):
    for j in range(len(number_of_files_Same)):
        if label_dataframe['With_zero'][i] == number_of_files_Same[j]:
            list_label.append(label_dataframe.iloc[i]['MGMT_value']) 
list_label = np.array(list_label)
s = 0; t = 0
for i in list_label:
    if i == 1:
        s = s + 1
    else:
        t = t + 1

In [ ]:
del csv_dataframe, s, t, label_dataframe
gc.collect()

In [ ]:
new_list_label = []
for i in range(len(list_label)):
    if list_label[i] == 1:
        a = np.ones(count_file_in_same_subfolder[i][0], dtype=int)
        new_list_label.append(a)             
        new_list_label.append(a)
        new_list_label.append(a)
        new_list_label.append(a)
    elif list_label[i] == 0:
        b = np.zeros(count_file_in_same_subfolder[i][0], dtype=int)
        new_list_label.append(b)             
        new_list_label.append(b)
        new_list_label.append(b)
        new_list_label.append(b)
        
new_list_label = np.array(new_list_label)

new_list_label_2 = []                 
for i in new_list_label:
    for j in i:
        new_list_label_2.append(j)
new_list_label_2 = np.array(new_list_label_2)

In [ ]:
del list_label, count_file_in_same_subfolder, a, b, i, j
gc.collect()

In [ ]:
import pydicom
from pydicom import dcmread
from pydicom.data import get_testdata_files

In [ ]:
%%time
training_set = []
for a in range(len(number_of_files_Same)):
    for b in os.listdir(path + "/" + number_of_files_Same[a]):     
        temp = []
        for c in os.listdir(path + "/" + number_of_files_Same[a] + "/" + b):
            temp.append(c)
            temp = sorted(temp)
        for d in range(len(temp)):
            ds = dcmread(path + "/" + number_of_files_Same[a] + "/" + b + "/" + temp[d], force=True)
            training_set.append(ds.pixel_array)

reshape_training_set = []
for i in range(len(training_set)):
    reshape_training_set.append(cv2.resize(training_set[i], (32, 32)))
reshape_training_set = np.array(reshape_training_set)

In [ ]:
del number_of_files_Same, training_set, a, b, c, d, i
gc.collect()

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 50: 
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

### CNN 

In [ ]:
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model       
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
X = copy.deepcopy(reshape_training_set)           
y = copy.deepcopy(new_list_label_2)

X_128_128_1 = []
for i in range(len(X)):
    q = np.reshape(X[i], (32,32,1))
    X_128_128_1.append(q)
X_128_128_1 = np.array(X_128_128_1)
print(X_128_128_1.shape)

In [ ]:
del X, reshape_training_set, new_list_label, new_list_label_2, i, q
gc.collect()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_128_128_1, y, test_size=0.2, random_state=0)
print('x_train:', X_train.shape)
print('label_train:', y_train.shape)
print('x_test:', X_test.shape)
print('label_test:', y_test.shape)

In [ ]:
del y, X_128_128_1
gc.collect()

In [ ]:
#16,(32),16,0.5,64,2 /// img:128*128 /// epoch:40, batch:all -> 0.89
#32,32,64,0.4,128,2 /// img:128*128 /// epoch:100, batch:all -> 0.89
#32,32,32,0.4,128,2 /// img:32*32   /// epoch:100, batch:all -> 0.84
#32,32,32,0.4,128,2 /// img:32*32   /// epoch:50, batch:all -> 0.80
#32,32,32,0.4,128,2 /// img:32*32   /// epoch:50, batch:150 -> 0.75
#16,(16),16,0.4,64,2
model = Sequential()

model.add(Conv2D(32, 3, padding="same", activation="relu", input_shape=(32,32,1)))
model.add(MaxPooling2D())

#model.add(Conv2D(32, 3, padding="same", activation="relu"))
#model.add(MaxPooling2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPooling2D())

model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(2, activation="softmax"))             

model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=100, batch_size=100, validation_data=(X_test,y_test)) 

In [ ]:
del X_train, X_test, y_train, y_test, temp, Adam, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Sequential
gc.collect()

In [ ]:
model.save('model.h5')